In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd
import string
import re

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


from utils import load_lda_datasets, load_test_dataset, load_model, get_stop_words

/home/lilith/.local/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


LOADING RAW DATA TREND-TEXT, LENGTH:  6737
LOADING CORPUS, LENGTH:  6737
LOADING DICTIONARY, LENGTH:  239733


ValueError: unsupported pickle protocol: 5

In [ ]:
DATA_DIR = "../Data"
TWEETS_PATH = os.path.join(DATA_DIR, 'tweets')
TREND_PATH = os.path.join(DATA_DIR, 'trends')
SAVE_PATH = os.path.join(DATA_DIR, 'save')
STATS_PATH = os.path.join(DATA_DIR, 'stats')
TOPICS_PATH = os.path.join(DATA_DIR, 'topics')

# LOAD DATA STRUCTURES TEST DATA

In [ ]:
dfs_train, trend_doc = utils.dfs_train, utils.trend_doc
# stemmed_dataset, corpus, dictionary = load_lda_datasets()

In [ ]:
stop_words = get_stop_words()
test_doc, stemmed_test, corpus_test = load_test_dataset()

In [ ]:
print(stemmed_test[0])
print(test_doc.loc[0].text)
print(corpus_test[:1])

# LOAD MODEL AND TOPIC LIST

In [ ]:
lda_model = load_model(10)

In [ ]:
target_doc = pd.read_csv(os.path.join(DATA_DIR, 'categories'), header=0)
target_doc.head(3)

In [ ]:
k_list_10 = ['Fashion','Art & Design','Sports','Technology','Politics','News','Entertainment','Books','Sports','Music']
k_topics_classified = dict(zip(np.arange(10),k_list_10))
print(k_topics_classified)

# VISUALIZE PRETRAINED MODEL

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=trend_doc.text):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)

        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences()

In [ ]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic['Dominant_Topic'] = df_dominant_topic['Dominant_Topic'].map(lambda x: k_topics_classified[x])
# Show
df_dominant_topic.head(10)

In [ ]:
trend_doc_topic = pd.concat([trend_doc, df_dominant_topic[['Dominant_Topic']]], axis=1)
trend_doc_topic.head(5)

# VISUALIZATION

In [ ]:
dfs_train.head(5)

In [ ]:
joined = pd.merge(trend_doc_topic[['trend','Dominant_Topic']],dfs_train, left_on='trend', right_on='trend')
joined

In [ ]:
topic_by_time = joined[['Dominant_Topic','trend_date','trend']].groupby(['trend_date','Dominant_Topic'])\
                ['trend'].apply(set).reset_index()

topic_by_time['Frequency'] = topic_by_time['trend'].apply(lambda x: len(x))
plot_df = topic_by_time.groupby(['trend_date']).apply(lambda x: x.nlargest(3, 'Frequency')).reset_index(drop=True)
plot_df

In [ ]:
import plotly.express as px

fig = px.bar(plot_df, x="trend_date", y="Frequency", color='Dominant_Topic', barmode='group',
             height=400, width=900 )
fig.show()

In [ ]:
topic_by_author = joined[['Dominant_Topic','trend_date', 'trend','author_id']].groupby(['trend_date','Dominant_Topic']).\
                    agg({'author_id': [set], 'trend':[set]}).reset_index()
topic_by_author.columns = ['trend_date', 'Dominant_Topic','author_id','trend']
topic_by_author['Frequency'] = topic_by_author['author_id'].apply(lambda x: len(x))
topic_by_author.drop(['author_id'], axis=1, inplace=True)

plot_df2 = topic_by_author.groupby(['trend_date']).apply(lambda x: x.nlargest(3, 'Frequency')).reset_index(drop=True)
plot_df2

In [ ]:
import plotly.express as px

fig = px.bar(plot_df2, x="trend_date", y="Frequency", color='Dominant_Topic', barmode='group',
             height=400, width=900 )
fig.show()